In [1]:
import pandas as pd
import numpy as np

In [2]:
# 엑셀 파일 경로 설정
file_path = "/Users/howsang/Documents/002.Study/001.project/001.internship/p_truck_delivery/data/winter_internship_prj_250122_v1.0.xlsx"  # 파일명을 실제 파일 이름으로 변경하세요.

# Excel 파일 읽기 (engine='openpyxl' 명시)
excel_data = pd.ExcelFile(file_path, engine='openpyxl')

# 모든 시트 이름 확인 (선택적으로 확인 가능)
print("Available sheets:", excel_data.sheet_names)

# 두 번째 시트의 데이터 읽기 (시트 인덱스는 0부터 시작)
df = pd.read_excel(file_path, sheet_name=1, engine='openpyxl')  # sheet_name=1은 두 번째 시트를 의미

# 데이터프레임 확인
print(df)

Available sheets: ['info', 'data_sample']
        GpsProvider           BookingID Market/Regular   \
0     CONSENT TRACK  MVCV0000927/082021          Market   
1           VAMOSYS  VCV00014271/082021         Regular   
2     CONSENT TRACK  VCV00014382/082021         Regular   
3           VAMOSYS  VCV00014743/082021         Regular   
4           VAMOSYS  VCV00014744/082021         Regular   
...             ...                 ...             ...   
6875          JTECH        WDSBKTP42751         Regular   
6876          JTECH        WDSBKTP43203         Regular   
6877          JTECH        WDSBKTP43021         Regular   
6878          JTECH        WDSBKTP42685         Regular   
6879          JTECH        WDSBKTP42858         Regular   

              BookingID_Date  vehicle_no  \
0    2020-08-17 14:59:01.000    KA590408   
1    2020-08-27 16:22:22.827  TN30BC5917   
2    2020-08-27 17:59:24.987  TN22AR2748   
3    2020-08-28 00:48:24.503  TN28AQ0781   
4    2020-08-28 01:23:19.243 

In [3]:
df.head()

,GpsProvider,BookingID,Market/Regular,BookingID_Date,vehicle_no,Origin_Location,Destination_Location,Org_lat_lon,Des_lat_lon,Data_Ping_time,...,TRANSPORTATION_DISTANCE_IN_KM,vehicleType,Minimum_kms_to_be_covered_in_a_day,Driver_Name,Driver_MobileNo,customerID,customerNameCode,supplierID,supplierNameCode,Material Shipped
0,CONSENT TRACK,MVCV0000927/082021,Market,2020-08-17 14:59:01.000,KA590408,"TVSLSL-PUZHAL-HUB,CHENNAI,TAMIL NADU","ASHOK LEYLAND PLANT 1- HOSUR,HOSUR,KARNATAKA","13.1550,80.1960","12.7400,77.8200",2020-08-24 00:05:09,...,320.0,NaN,NaN,NaN,NaN,ALLEXCHE45,Ashok leyland limited,VIJEXHOSR7,VIJAY TRANSPORT,BRACKET / GRAB HANDLE
1,VAMOSYS,VCV00014271/082021,Regular,2020-08-27 16:22:22.827,TN30BC5917,"DAIMLER INDIA COMMERCIAL VEHICLES,KANCHIPURAM,...","DAIMLER INDIA COMMERCIAL VEHICLES,KANCHIPURAM,...","12.8390,79.9540","12.8390,79.9540",2020-08-28 12:40:28,...,103.0,NaN,NaN,RAMESH,NaN,DMREXCHEUX,Daimler india commercial vehicles pvt lt,VJLEXSHE09,VJ LOGISTICS,ZB MODEL PLATE / 3143
2,CONSENT TRACK,VCV00014382/082021,Regular,2020-08-27 17:59:24.987,TN22AR2748,"LUCAS TVS LTD-PONDY,PONDY,PONDICHERRY","LUCAS TVS LTD-PONDY,PONDY,PONDICHERRY","11.8710,79.7390","11.8710,79.7390",2020-08-28 09:05:09,...,300.0,NaN,NaN,GIRI,NaN,LUTGCCHE06,Lucas tvs ltd,GSTEXLAK1Q,G.S. TRANSPORT,LETTERING / FUSO
3,VAMOSYS,VCV00014743/082021,Regular,2020-08-28 00:48:24.503,TN28AQ0781,"DAIMLER INDIA COMMERCIAL VEHICLES,KANCHIPURAM,...","DAIMLER INDIA COMMERCIAL VEHICLES,KANCHIPURAM,...","12.8390,79.9540","12.8390,79.9540",2020-08-28 12:40:31,...,61.0,NaN,NaN,RAVI,NaN,DMREXCHEUX,Daimler india commercial vehicles pvt lt,ARVEXNAM09,ARVINTH TRANSPORT,LU STRUT RA / RADIUS ROD
4,VAMOSYS,VCV00014744/082021,Regular,2020-08-28 01:23:19.243,TN68F1722,"LUCAS TVS LTD-PONDY,PONDY,PONDICHERRY","LUCAS TVS LTD-PONDY,PONDY,PONDICHERRY","11.8720,79.6320","11.8720,79.6320",2020-08-28 12:40:29,...,240.0,NaN,NaN,TAMIL,NaN,LUTGCCHE06,Lucas tvs ltd,SRTEXKOR96,SR TRANSPORTS,WISHBONE / V ROD/HDT
